In [2]:
import numpy as np
import scipy as sp
import scipy.special
import scipy.linalg
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd

In [11]:
# Load test data
Y = np.loadtxt('Neuron2193.txt')
dN = Y
print dN.shape

(45, 2000)


In [4]:
def Do_Kalman_Likelihood_Bernoulli_LaplaceMAP(dN, sigma2e, tol=1e-8, trials=1.):
    """MAP solution, inverse covariance matrix, and marginal loglikelihood of state-space model
    computed using Laplace approximation around MAP state.

    :param dN: Observations (K,)
    :param sigma2e: Variance of process noise
    :param tol: Convergence criterion on the gradient of the log-likelihood
    :param trials: Number of trials for binomial observations (1 for Bernoulli)
    :return: x_map, U, marginal_loglikelihood, joint_loglikelihood
    """
    x = np.zeros(dN.shape)
    dN = dN.astype(float)
    while True:
        # Build gradient of joint
        d2x = np.convolve(x, [-1, 2, -1])[1:-1]
        d2x[-1] -= x[-1]
        G = -dN + trials * (1. / (1. + np.exp(-x))) + d2x / sigma2e
        # Build Hessian of joint
        D = trials / (np.exp(x) + 2. + np.exp(-x)) + 2. / sigma2e
        D[-1] -= 1. / sigma2e
        B = -np.ones(len(D)) / sigma2e
        B[0] = 0.
        U = sp.linalg.cholesky_banded((B, D), lower=False)
        # Check convergence
        if np.dot(G, G) < tol:
            x_map = x
            break
        # Update estimate of map
        x -= sp.linalg.cho_solve_banded([U, False], G)

    # Compute joint and marginal probabilities
    joint_loglikelihood = (np.sum(np.log(sp.special.binom(trials, dN)) + dN * x_map - trials * np.log(1 + np.exp(x_map))) -
                           .5 * ((np.sum(np.diff(x_map)**2) + x_map[0]**2) / sigma2e + len(dN) * np.log(2*np.pi*sigma2e)))
    marginal_loglikelihood = len(dN)/2. * np.log(2*np.pi) + joint_loglikelihood - np.sum(np.log(U[-1]))
    return x_map, U, marginal_loglikelihood, joint_loglikelihood

In [5]:
def cov_from_chol_precision(U):
    """Given the Cholesky factorization (U) of the posterior precision matrix (J), with U^t * U = J,
    return the tridiagonal part of the covariance matrix.

    :param U: Cholesky factorization (U) of J, given as [0, A; D] where A is the upper diagonal and D the main diagonal
    :return: Cov_tri: Tridiagonal part of the covariance matrix returned as [0, C_i,i+1; C_ii; C_i+1,i, 0]
    """
    assert(U.shape[0] == 2 and U[0,0] == 0)
    A, D = U # Unpack matrix into first (above) diagonal and diagonal
    Cov_tri = np.zeros_like(U)
    C, V = Cov_tri # Obtain _views_ into the first diagonal and diagonal
    # Compute last element of diagonal
    V[-1] = 1. / (D[-1] ** 2)
    # Recursively compute other elements of main diagonal and first diagonal
    for i in range(len(D)-1, 0, -1):
        iD = 1. / D[i-1]
        iDA = iD * A[i]
        N = -iDA * V[i]
        C[i] = N
        V[i-1] = iD ** 2 - N * iDA
    return Cov_tri

In [6]:
def Cluster_Laplace(dN, sigma2e_init, prior_clusters, trials=1., verbose=True):
    assert(len(sigma2e_init) == len(prior_clusters))
    C = len(prior_clusters)
    M, K = dN.shape
    posterior_clusters = np.tile(prior_clusters, [M,1]).T
    sigma2e = sigma2e_init
    expect_log_likel_old = np.NaN
    while True:
        sigma2e_new = np.zeros((C, M))
        log_prob_dNi_given_c = np.zeros((C, M))
        for i, dN_i in enumerate(dN):
            for c, sigma2e_c in enumerate(sigma2e):
                x_map, U, marginal_loglik, _ = Do_Kalman_Likelihood_Bernoulli_LaplaceMAP(dN_i, sigma2e_c, trials=trials)
                Cov_tri = cov_from_chol_precision(U)
                sigma2e_new[c,i] = (np.sum(Cov_tri[1]) + np.dot(x_map, x_map) # E[x_k^2]
                                   + np.sum(Cov_tri[1,:-1]) + np.dot(x_map[:-1], x_map[:-1]) # E[x_{k-1}^2]
                                   - 2 * np.sum(Cov_tri[0]) - 2 * np.dot(x_map[1:], x_map[:-1])) / K # E[x_{k-1} * x_k]
                log_prob_dNi_given_c[c,i] = marginal_loglik
        expect_log_likel = np.sum(posterior_clusters * log_prob_dNi_given_c)
        if verbose:
            print(expect_log_likel, sigma2e)
        if (abs(expect_log_likel - expect_log_likel_old) < 1e-6 * abs(expect_log_likel_old)):
            break
        for c, sigma2e_new_c in enumerate(sigma2e):
            if sigma2e_new_c < 0.05:
                break
        expect_log_likel_old = expect_log_likel
        sigma2e = np.sum(posterior_clusters * sigma2e_new, axis=1) / np.sum(posterior_clusters, axis=1)
        posterior_clusters = np.exp(log_prob_dNi_given_c - np.max(log_prob_dNi_given_c, axis=0)) * prior_clusters[:,None]
        posterior_clusters /= np.sum(posterior_clusters, axis=0)
        prior_clusters = np.sum(posterior_clusters, axis=1) / np.sum(posterior_clusters, axis=None)
    return sigma2e, prior_clusters, posterior_clusters, expect_log_likel

In [18]:
sigma2e_init = np.array([0.09, 0.07])
prior_clusters = np.array([0.5, 0.5])
sigma2e, prior_clusters, posterior_clusters, expect_log_likel = Cluster_Laplace(dN, sigma2e_init, prior_clusters, trials=1)

(-2596.8399760229031, array([ 0.09,  0.07]))
(-2592.2045869781032, array([ 0.08995541,  0.07002759]))
(-2590.7926510852203, array([ 0.08994668,  0.07001673]))
(-2589.7609437592687, array([ 0.08993048,  0.06999986]))
(-2589.0386671364058, array([ 0.0899081 ,  0.06997778]))
(-2588.5431097995579, array([ 0.08988072,  0.06995146]))
(-2588.2067972620634, array([ 0.08984943,  0.06992182]))
(-2587.980279396299, array([ 0.08981514,  0.06988961]))
(-2587.8291639833301, array([ 0.08977859,  0.06985546]))
(-2587.7300665081525, array([ 0.08974036,  0.06981987]))
(-2587.6671733615681, array([ 0.08970091,  0.06978323]))
(-2587.6297427631748, array([ 0.08966058,  0.06974584]))
(-2587.6104061878727, array([ 0.08961963,  0.06970793]))
(-2587.6040473775251, array([ 0.08957826,  0.06966965]))
(-2587.6070692310082, array([ 0.08953661,  0.06963116]))
(-2587.6169129986624, array([ 0.08949481,  0.06959253]))
(-2587.6317397952457, array([ 0.08945293,  0.06955384]))
(-2587.6502166838759, array([ 0.08941103,  0

KeyboardInterrupt: 

In [17]:
print('---- Expectation of log_likelihood -----')
print(expect_log_likel)
print('---- Sigma2e -----')
print(sigma2e)
print('---- Prior clusters (estimated) -----')
print(prior_clusters)
print('---- Posterior clusters -----')
print(posterior_clusters)

---- Expectation of log_likelihood -----
-2596.27218369
---- Sigma2e -----
[ 0.0789033   0.04969628]
---- Prior clusters (estimated) -----
[  1.00000000e+000   8.06606304e-295]
---- Posterior clusters -----
[[  1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+000   1.00000000e+000
    1.00000000e+000   1.00000000e+000   1.00000000e+0

In [ ]:
for i in xrange(posterior_clusters.shape[0]):
    print np.where(posterior_clusters[i,:] > 0.95)

In [ ]:
sigma2e_init = np.array([0.5, 0.6, 0.7])
prior_clusters = np.array([0.33, 0.33, 0.34])
sigma2e, prior_clusters, posterior_clusters, expect_log_likel = Cluster_Laplace(dN, sigma2e_init, prior_clusters, trials=trial1)
print('---- Expectation of log_likelihood -----')
print(expect_log_likel)
print('---- Sigma2e -----')
print(sigma2e)
print('---- Prior clusters (estimated) -----')
print(prior_clusters)
print('---- Posterior clusters -----')
print(posterior_clusters)

In [ ]:
kkk = pd.read_csv('cluster_train_data.csv', header=None)
print kkk.shape